In [31]:
import yfinance as yf
import pymysql
import datetime 
import pandas as pd 
from dateutil.relativedelta import relativedelta
from yahoo_earnings_calendar import YahooEarningsCalendar
yec = YahooEarningsCalendar()

In [3]:
# 1. DB 연결
def connect_db():
    conn = pymysql.connect(host="localhost", user="root", password="4team123!",
                           db="stockDB", charset="utf8")  
    return conn

today = datetime.datetime.now().strftime('%Y%m%d') 
conn = connect_db()
curs = conn.cursor() 

In [5]:
# 이미 입력된 TICKER 리스트 
sql = "SELECT DISTINCT SYMBOL FROM USA_STOCK_INFO" 
curs.execute(sql) 
stock_list = [item[0] for item in curs.fetchall()] 
conn.commit() 
print(len(stock_list))

7525


In [7]:
# 입력된 ticker의 최근 실적발표일을 리턴 
def get_pre_earning_date(ticker): 
    pre_earning_date = [] 
    try: 
        history = si.get_earnings_history(ticker) 
        before_now = [x['startdatetime'][0:10].replace('-','') for x in history if x['startdatetime'][0:10].replace('-','') < today] 
        pre_earning_date.append(before_now[0])         
    except: 
        pre_earning_date.append('') 
    return pre_earning_date

In [8]:
def get_next_earning_date(ticker): 
    next_earning_date = [] 
    try: 
        next_earning_date.append(si.get_next_earnings_date(ticker).strftime('%Y%m%d')) 
    except: 
        next_earning_date.append('') 
    return next_earning_date

In [ ]:
sql = """update USA_STOCK_INFO set PRE_EARNING_DATE = %s , NEXT_EARNING_DATE = %s where symbol = %s""" 

# 반복문 돌면서 INSERT 
for i, ticker in enumerate(stock_list): 
    try: 
        earning_date_info = get_pre_earning_date(ticker) 
        earning_date_info = earning_date_info + get_next_earning_date(ticker) 
        earning_date_info = earning_date_info + [ticker] 
#         curs.execute(sql, earning_date_info) 
#         conn.commit() 
    except Exception as e: 
        print(i, ' 번 째 오류 발생 : ', ticker, ' 오류:', str(e))

# DB 데이터 로딩

In [28]:
earning_date_info

['', '', 'GLO RT WI']

In [14]:
cur = conn.cursor() # 2. 커서 생성 (트럭, 연결로프)
sql = "SELECT * FROM `usa_stock_info`"
cur.execute(sql)

rows = cur.fetchall()

In [21]:
dataList = list(rows)

df = pd.DataFrame(
    dataList, columns=['index', 'Symbol', 'Name', 'Industry', 'IndustryCode'])

In [27]:
df

,index,Symbol,Name,Industry,IndustryCode
0,0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",571060
1,1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,572010
2,2,AMZN,Amazon.com Inc,다양한 소매업,534020
3,3,FB,Facebook Inc Class A,소프트웨어 및 IT서비스,572010
4,4,GOOGL,Alphabet Inc Class A,소프트웨어 및 IT서비스,572010
...,...,...,...,...,...
7524,292,IPB,Merrill LYNCH Depositor Inc Indexplus Trust Se...,은행,551010
7525,293,VHAQ RT,Viveon Health Acquisition Corp,기타 금융업,556010
7526,294,GLQ RT WI,Clough Global Equity Fund,다양한 금융서비스,555010
7527,295,GLV RT WI,Clough Global Dividend and Income Fund,다양한 금융서비스,555010


# 실적발표일

In [ ]:
# Execute the to_sql for writting DF into SQL
df_stock.to_sql('usa_stock_info', engine, if_exists='append', index=False)  


In [91]:
# Returns the next earnings date of BOX in Unix timestamp
def get_next_earning_date(ticker):       
    return yec.get_next_earnings_date(ticker)

In [92]:
get_next_earning_date('AAPL')

1627430400

# 배당정보

In [63]:
import finnhub

# Setup client
# finnhub_client = finnhub.Client(api_key="c3dg1raad3icrjj6ha80")
finnhub_client = finnhub.Client(api_key="sandbox_c3dg1raad3icrjj6ha8g")

token='c3dg1raad3icrjj6ha80'
# token='sandbox_c3dg1raad3icrjj6ha8g'
api_key='sandbox_c3dg1raad3icrjj6ha8g'
token='sandbox_c3dg1raad3icrjj6ha8g'
url = f'https://finnhub.io/api/v1/stock/symbol?exchange=US&token={api_key}'
hist_url = 'https://finnhub.io/api/v1/stock/metric?'

import requests
def get_dividends(symbol):
    global token
    url = f'https://finnhub.io/api/v1/stock/dividend2?symbol={symbol}&token={token}'
    r = requests.get(url)
    return r.json()
#     print(r.json())

In [98]:
ret = get_dividends('AAPL')

In [99]:
dividends_date = ret['data'][-1]['exDate']

In [100]:
print(dividends_date)

2021-05-07


In [101]:
recent = ret['data'][-5:]

In [109]:
sum = 0
recents = ret['data'][-5:]
freq = len(recents)
for div in recents:
    sum += div['amount']
    print(div['amount'])
    
print(sum)
avg = sum/freq
print(avg)

0.82
0.82
0.205
0.205
0.22
2.27
0.454


In [74]:
this_div_date = ret['data'][-1]['exDate']
this_div_date

'2021-05-07'